In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import numpy as np
import tabula
import os

**DATA EXTRACTION**

In [2]:
# WEB SCRAPPING TABLE (df_salario):

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service = service)
driver.get('https://portal.trt3.jus.br/internet/servicos/valores/salario-minimo')
url = driver.find_element(By.XPATH,'//*[@id="parent-fieldname-text"]/div/table')
html = url.get_attribute('outerHTML')
driver.quit()
df_salario = pd.read_html(html)[0]

In [3]:
# WEB SCRAPPING TABLE FROM PDF (margens_2013):

margens_2013 = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2013.pdf', pages = 'all')
margens_2013 = pd.concat(margens_2013)

In [4]:
# WEB SCRAPPING TABLE FROM PDF (margens_2023):

margens = tabula.read_pdf(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\margens_2023.pdf', pages = 'all')
margens_2023 = pd.concat(margens)

In [5]:
#READING XLSX FILES IN DATAFRAME (brasil):

brasil = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\brasil_semanal.xlsx')

In [6]:
#READING XLSX FILES IN DATAFRAME (estados):

estados = pd.read_excel(r'C:\Users\pedro\OneDrive\Área de Trabalho\GAS\estados_semanal.xlsx')

*EXPLORING AND CLEANING THE DATA*

**** DATAFRAME brasil****

In [7]:
#EXPLORING THE brasil DATAFRAME:
brasil.head()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,SUPERINTENDÊNCIA DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA DE LEVANTAMENTO DE PREÇOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RELATÓRIO DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#EXPLORING THE brasil DATAFRAME:
brasil.tail()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
3295,2023-03-26 00:00:00,2023-04-01 00:00:00,GASOLINA COMUM,4312,R$/l,5.48,0.35,4.27,7.19,-,0.064,-,-,-,-,-
3296,2023-03-26 00:00:00,2023-04-01 00:00:00,GLP,3944,R$/13kg,107.33,11.693,74.99,155,-,0.109,-,-,-,-,-
3297,2023-03-26 00:00:00,2023-04-01 00:00:00,GNV,353,R$/m³,4.5,0.543,3.54,6.59,-,0.12,-,-,-,-,-
3298,2023-03-26 00:00:00,2023-04-01 00:00:00,OLEO DIESEL,1953,R$/l,5.8,0.397,5.06,7.69,-,0.068,-,-,-,-,-
3299,2023-03-26 00:00:00,2023-04-01 00:00:00,OLEO DIESEL S10,3104,R$/l,5.86,0.391,5.09,8.49,-,0.067,-,-,-,-,-


In [9]:
#EXPLORING THE brasil DATAFRAME:
brasil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3300 entries, 0 to 3299
Data columns (total 16 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP  3297 non-null   object
 1   Unnamed: 1                                                         3284 non-null   object
 2   Unnamed: 2                                                         3284 non-null   object
 3   Unnamed: 3                                                         3284 non-null   object
 4   Unnamed: 4                                                         3284 non-null   object
 5   Unnamed: 5                                                         3284 non-null   object
 6   Unnamed: 6                                                         3284 non-null   object
 7   Unnamed: 7                       

In [10]:
#PERFORMING TREATMENT IN DATAFRAME brasil USING DE FUNCTION BELOW:

def limpar_df_br(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime
    df = df.rename(columns = {'AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP':'DATA INICIAL','Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'PRODUTO','Unnamed: 3':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 4':'UNIDADE DE MEDIDA','Unnamed: 5':'PREÇO MÉDIO REVENDA','Unnamed: 6':'DESVIO PADRÃO REVENDA','Unnamed: 7':'PREÇO MÍNIMO REVENDA','Unnamed: 8':'PREÇO MÁXIMO REVENDA','Unnamed: 9':'MARGEM MÉDIA REVENDA','Unnamed: 10':'COEF DE VARIAÇÃO REVENDA','Unnamed: 11':'PREÇO MÉDIO DISTRIBUIÇÃO','Unnamed: 12':'DESVIO PADRÃO DISTRIBUIÇÃO','Unnamed: 13':'PREÇO MÍNIMO DISTRIBUIÇÃO','Unnamed: 14':'PREÇO MÁXIMO DISTRIBUIÇÃO','Unnamed: 15':'COEF DE VARIAÇÃO DISTRIBUIÇÃO'})
    df_1 = df.drop(index=range(0, 17)).drop(columns=['DATA INICIAL', 'DESVIO PADRÃO REVENDA', 'MARGEM MÉDIA REVENDA','COEF DE VARIAÇÃO REVENDA', 'PREÇO MÉDIO DISTRIBUIÇÃO','DESVIO PADRÃO DISTRIBUIÇÃO',	'PREÇO MÍNIMO DISTRIBUIÇÃO','PREÇO MÁXIMO DISTRIBUIÇÃO','COEF DE VARIAÇÃO DISTRIBUIÇÃO'])
    df_limpo = df_1.loc[df_1['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo.index.name = 'DATA'
    return df_limpo


In [11]:
# NEW DATAFRAME TREATED (glp_br):
glp_br = limpar_df_br(brasil)

***DATAFRAME estado***

In [12]:
#EXPLORING THE (estados) DATAFRAME:
estados.head()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,SUPERINTENDÊNCIA DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SISTEMA DE LEVANTAMENTO DE PREÇOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RELATÓRIO DE DEFESA DA CONCORRÊNCIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#EXPLORING THE (estados) DATAFRAME:
estados.tail()

,"AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
82865,2023-03-26 00:00:00,2023-04-01 00:00:00,NORTE,RORAIMA,OLEO DIESEL S10,14,R$/l,7,0.041,6.97,7.13,-,0.006,-,-,-,-,-
82866,2023-03-26 00:00:00,2023-04-01 00:00:00,SUL,SANTA CATARINA,OLEO DIESEL S10,133,R$/l,5.86,0.261,5.31,6.79,-,0.045,-,-,-,-,-
82867,2023-03-26 00:00:00,2023-04-01 00:00:00,SUDESTE,SAO PAULO,OLEO DIESEL S10,785,R$/l,5.86,0.402,5.09,8.49,-,0.069,-,-,-,-,-
82868,2023-03-26 00:00:00,2023-04-01 00:00:00,NORDESTE,SERGIPE,OLEO DIESEL S10,24,R$/l,5.87,0.303,5.57,6.49,-,0.052,-,-,-,-,-
82869,2023-03-26 00:00:00,2023-04-01 00:00:00,NORTE,TOCANTINS,OLEO DIESEL S10,41,R$/l,5.61,0.307,5.25,6.9,-,0.055,-,-,-,-,-


In [14]:
#EXPLORING THE (estados) DATAFRAME:
estados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82870 entries, 0 to 82869
Data columns (total 18 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP  82867 non-null  object
 1   Unnamed: 1                                                         82854 non-null  object
 2   Unnamed: 2                                                         82854 non-null  object
 3   Unnamed: 3                                                         82854 non-null  object
 4   Unnamed: 4                                                         82854 non-null  object
 5   Unnamed: 5                                                         82854 non-null  object
 6   Unnamed: 6                                                         82854 non-null  object
 7   Unnamed: 7                     

In [15]:
#PERFORMING TREATMENT IN DATAFRAME estados USING DE FUNCTION BELOW:

def limpar_df_estado(df):
    '''This function performs the following steps on any project dataframe: cleans columns and rows that are not relevant to the analysis;
    performs substitutions in order to improve aesthetics and finally filters the results only from GLP.'''
    import pandas as pd
    import numpy as np
    from datetime import datetime   
    df = df.drop(index=range(0, 17)).drop(columns = ['AGÊNCIA NACIONAL DO PETRÓLEO, GÁS NATURAL E BIOCOMBUSTÍVEIS - ANP', 'Unnamed: 8', 'Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17'])
    estados_limpo = df.rename(columns = {'Unnamed: 1':'DATA FINAL', 'Unnamed: 2':'REGIÃO','Unnamed: 3':'ESTADO','Unnamed: 4':'PRODUTO','Unnamed: 5':'NÚMERO DE POSTOS PESQUISADOS','Unnamed: 6':'UNIDADE DE MEDIDA','Unnamed: 7':'PREÇO MÉDIO REVENDA','Unnamed: 9':'PREÇO MÍNIMO REVENDA','Unnamed: 10':'PREÇO MÁXIMO REVENDA'})
    df_limpo = estados_limpo.loc[estados_limpo['PRODUTO'] == 'GLP']
    df_limpo = df_limpo.replace('-', 'Não Informado').replace('R$/13Kg', '13Kg').replace('R$/13kg', '13Kg')
    df_limpo['DATA FINAL'] = pd.to_datetime(df_limpo['DATA FINAL'])
    df_limpo['DATA FINAL'] = df_limpo['DATA FINAL'].apply(lambda x: datetime.strftime(x, '%b-%Y'))
    df_limpo = df_limpo.set_index('DATA FINAL')
    df_limpo.index.name = 'DATA'
    return df_limpo


In [16]:
# NEW DATAFRAME TREATED (glp_regioes):
glp_regioes = limpar_df_estado(estados)